In [12]:
import keras
from keras import backend as K
from keras.layers import Input, Lambda, Dense, Conv2D, Conv2DTranspose, MaxPool2D, Flatten, Reshape
from keras.models import Model
from keras import metrics
from keras.datasets import mnist
import tensorflow as tf
from tensorflow.python import debug as tf_debug
import numpy as np

In [13]:
batch_size = 128
epochs = 50
image_size = (28,28,1)
latent_dimension = 3 ## to view representation clusters in 3 dimensions

In [14]:
## defining the input for mnist images
input_image = Input(shape=image_size)

In [28]:
## defining the inference network
## this is the network that will produce a latent space representation of the original image
## 5 layer convolutional network
encoder = Conv2D(16, (3,3), activation='relu', padding='same', name="encoder_conv1")(input_image)
encoder = MaxPool2D((2,2), padding="same", name="encoder_pool1")(encoder)
encoder = Conv2D(8, (3,3), activation='relu', padding='same', name="encoder_conv2")(encoder)
encoder = MaxPool2D((2,2), padding="same", name="encoder_pool2")(encoder)
encoder = Conv2D(4, (3,3), activation='relu', padding='same', name="encoder_conv3")(encoder)
encoder_shape = K.int_shape(encoder)
encoder = Flatten()(encoder) ## turns output to size of (None, 196)
# testing without further reduction of dimensions
encoder = Dense(32, activation='relu')(encoder)

z_mean = Dense(latent_dimension, name="z_mean")(encoder)
z_var = Dense(latent_dimension, name="z_var")(encoder)

In [22]:
## defining the sampling method for the generator network
def normal_sample(args):
    z_mean, z_log_var = args
    epsilon = K.random_normal(shape=(K.shape(z_mean)[0], latent_dimension),
                              mean=0., stddev=1.)
    return z_mean + K.exp(z_log_var) * epsilon
z = Lambda(normal_sample, name="z_sample")([z_mean, z_var])
sample_shape = K.int_shape(z)

In [38]:
## defining the generator network
## this is the network that takes the latent space representation and upsamples it to create a reconstruction
## 5 layer convolutional upsampling network
decoder_input = Input(sample_shape[1:])
decoder = Dense(np.prod(encoder_shape[1:]), activation="relu")(decoder_input)
decoder = Reshape(encoder_shape[1:])(decoder)
print(decoder.shape)
## figure out the shaping problem
decoder  = Conv2DTranspose(32, 3,
                           padding='same', activation='relu',
                           strides=(2, 2))(decoder)
print(decoder.shape)
decoder = Conv2D(1, 3, padding='same', activation='sigmoid')(decoder)
print(decoder.shape)
decoder = Model(decoder_input, decoder)

(?, 7, 7, 4)
(?, ?, ?, 32)
(?, ?, ?, 1)


In [35]:
## variational layer for reconstruction loss
class CustomVariationalLayer(keras.layers.Layer):

    def vae_loss(self, x, z_decoded):
        x = K.flatten(x)
        z_decoded = K.flatten(z_decoded)
        xent_loss = keras.metrics.binary_crossentropy(x, z_decoded)
        kl_loss = -5e-4 * K.mean(
            1 + z_var - K.square(z_mean) - K.exp(z_var), axis=-1)
        return K.mean(xent_loss + kl_loss)

    def call(self, inputs):
        x = inputs[0]
        z_decoded = inputs[1]
        loss = self.vae_loss(x, z_decoded)
        self.add_loss(loss, inputs=inputs)
        # We don't use this output.
        return x

In [36]:
## reconstruct the sample taken from the latent space
decoded_sample = decoder(z) ## generated sample
y = CustomVariationalLayer()([input_image, decoded_sample]) ## reconstruction loss applied

In [37]:
from keras.datasets import mnist

vae = Model(input_image, y)
vae.compile(optimizer='rmsprop', loss=None)
vae.summary()

# Train the VAE on MNIST digits
(x_train, _), (x_test, y_test) = mnist.load_data()

x_train = x_train.astype('float32') / 255.
x_train = x_train.reshape(x_train.shape + (1,))
x_test = x_test.astype('float32') / 255.
x_test = x_test.reshape(x_test.shape + (1,))

vae.fit(x=x_train, y=None,
        shuffle=True,
        epochs=10,
        batch_size=batch_size,
       )

/Users/ShishirJakati/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Output "custom_variational_layer_5" missing from loss dictionary. We assume this was done on purpose, and we will not be expecting any data to be passed to "custom_variational_layer_5" during training.
  after removing the cwd from sys.path.


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 28, 28, 1)    0                                            
__________________________________________________________________________________________________
encoder_conv1 (Conv2D)          (None, 28, 28, 16)   160         input_3[0][0]                    
__________________________________________________________________________________________________
encoder_pool1 (MaxPooling2D)    (None, 14, 14, 16)   0           encoder_conv1[0][0]              
__________________________________________________________________________________________________
encoder_conv2 (Conv2D)          (None, 14, 14, 8)    1160        encoder_pool1[0][0]              
__________________________________________________________________________________________________
encoder_po

InvalidArgumentError: Incompatible shapes: [25088] vs. [100352]
	 [[Node: custom_variational_layer_5/logistic_loss/mul = Mul[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"](custom_variational_layer_5/Log, custom_variational_layer_5/Reshape)]]

Caused by op 'custom_variational_layer_5/logistic_loss/mul', defined at:
  File "/Users/ShishirJakati/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/ShishirJakati/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/ShishirJakati/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/ShishirJakati/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/ShishirJakati/anaconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance(